In [1]:
#!/usr/bin/python
# -*- coding: latin-1 -*-

import os, sys, inspect
current_folder = os.path.realpath(os.path.abspath(os.path.split(inspect.getfile(inspect.currentframe()))[0]))
folder_parts = current_folder.split(os.sep)
pywikibot_folder = os.sep.join(folder_parts[0:-1])
if current_folder not in sys.path:
    sys.path.insert(0, current_folder)
if pywikibot_folder not in sys.path:
    sys.path.insert(0, pywikibot_folder)

import pywikibot as pb
from pywikibot.specialbots import UploadRobot

from bs4 import BeautifulSoup
import urllib
from mako.template import Template

In [2]:
commons_site = pb.Site("commons", "commons")

In [5]:
import urllib2

url = 'https://diario.madrid.es/blog/notas-de-prensa/carmena-propone-un-debate-directo-entre-las-ciudades-y-la-onu/'
user_agent = 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Gecko/20100101 Firefox/40.1'
headers = { 'User-Agent' : user_agent }
excluded = []
categories = ['Manuela Carmena in 2017', 'September 2017 in New York', 'United Nations Human Settlements Programme']
cat_string = '\n'.join(['[[Category:'+i+']]' for i in categories])

req = urllib2.Request(url, headers=headers)
response = urllib2.urlopen(req).read()
soup = BeautifulSoup(response, 'html.parser')

In [6]:
import calendar
title = soup.find_all("h1", class_="post-title")[0].get_text().strip().replace('  ', ' ')
date = '-'.join(soup.find_all("div", class_="post-date")[0].get_text().strip().split('/')[::-1])
#date = "2017-07-07"
year = date.split('-')[0]
month = calendar.month_name[int(date.split('-')[1])]
body = soup.find_all("div", class_="post-content")
p_description = body[0].find_all("p")
#print p_description
for p in p_description :
    if len(p.get_text()) > 10:
        description = p.get_text()
        break
#description=u'Una delegación del Ayuntamiento de Madrid asiste a la manifestación de Barcelona en contra de los atentados terroristas, presidida por Manuela Carmena, y con representación de todos los grupos políticos municipales. Esta representación municipal ha estado compuesta por la portavoz del gobierno municipal, Rita Maestre: el delegado de Coordinación Territorial y Cooperación Público-Social y concejal del Grupo Municipal Ahora Madrid, Nacho Murgui; la concejala y delegada del portavoz del Grupo Municipal Partido Popular, Isabel Rosell; el concejal y delegado de la portavoz del Grupo Municipal Socialista, Ignacio de Benito y la portavoz del Grupo Municipal Ciudadanos-Partido de la Ciudadanía, Begoña Villacís.\n"Madrid como ciudad está aquí, con Barcelona, por solidaridad y por la necesidad de expresar la protesta que significan los actos de esta barbarie, para hacer posible que no se vuelvan a repetir", ha dicho la alcaldesa a su llegada a la capital catalana.'

In [7]:
template = u"""=={{int:filedesc}}==
{{Information
|description={{es|1=${description}}}
|date=${date}
|source=[${url} Diario de Madrid - ${title}]
|author=[https://diario.madrid.es/ Diario de Madrid]
|permission=[https://diario.madrid.es/contenidos-libres/ License information for all contents in diario.madrid.es]
|other versions=
}}

=={{int:license-header}}==
{{Diario de Madrid}}

[[Category:Files uploaded by User:Discasto]]
[[Category:Images from Ayuntamiento de Madrid (to classify)]]
[[Category:${month} ${year} in Madrid]]
${cat_string}"""
vars = {
    "url": url,
    "description": description,
    "year": year,
    "month": month,
    "date": date,
    "title": title,
    "cat_string": cat_string
}
t = Template(template)
_text = t.render(**vars)
_text

u'=={{int:filedesc}}==\n{{Information\n|description={{es|1=La alcaldesa de Madrid, Manuela Carmena, ha participado hoy en un foro de la Asamblea General de la Organizaci\xf3n de Naciones Unidas, para desarrollar la Nueva Agenda Urbana y el papel que tienen las autoridades municipales en este \xe1mbito. Carmena ha reivindicado el papel protagonista de los gobiernos locales en la implementaci\xf3n de la agenda internacional y ha propuesto la creaci\xf3n de una agencia que permita el di\xe1logo directo entre los gobiernos locales y la ONU. En este sentido, la alcaldesa ha se\xf1alado que \u201cno podemos solo hablar de planificaci\xf3n si no hablamos de derechos humanos. No podemos soportar que haya ciudades y pa\xedses en los que el n\xfamero de personas privadas de derechos son millones\u201d. Carmena adem\xe1s ha subrayado que \u201ces importante que se cree una verdadera cultura de la ciudad. Nos falta cultura de la ciudad\u201d, ha insistido.}}\n|date=2017-09-06\n|source=[https://dia

In [8]:
counter = 0
for i, image in enumerate(soup.find_all("div", class_="gallery-icon")) :
    print image
    if i in excluded:
        continue
    href = urllib2.quote(image.a["href"].encode('utf-8'), ':/')
    image_name = u'%s (%02d).jpg' % (title, counter+1)
    image_name = image_name.replace(':', ' -').replace('  ', ' ')
    img_request = urllib2.Request(href, headers=headers)
    img_data = urllib2.urlopen(img_request).read()
    image_path = r'D:\\tmp\\' + href.split('/')[-1]
    print image_path
    f = open(image_path, 'wb')
    f.write(img_data)
    f.close()
    bot = UploadRobot(image_path,
                  description = _text,
                  useFilename=image_name,
                  keepFilename=True,
                  verifyDescription=False,
                  ignoreWarning=True,
                  targetSite = commons_site)
    #print image_name
    bot.run()
    os.remove(image_path)
    counter += 1

<div class="gallery-icon landscape"> <a href="https://diario.madrid.es/wp-content/uploads/2017/09/Con-alcalde-de-NY.jpg"><img alt="" class="attachment-un_gallery size-un_gallery" height="400" src="https://diario.madrid.es/wp-content/uploads/2017/09/Con-alcalde-de-NY-750x400.jpg" width="750"/></a></div>
D:\\tmp\\Con-alcalde-de-NY.jpg


The suggested description is:
=={{int:filedesc}}==
{{Information
|description={{es|1=La alcaldesa de Madrid, Manuela Carmena, ha participado hoy en un foro de la Asamblea General de la Organización de Naciones Unidas, para desarrollar la Nueva Agenda Urbana y el papel que tienen las autoridades municipales en este ámbito. Carmena ha reivindicado el papel protagonista de los gobiernos locales en la implementación de la agenda internacional y ha propuesto la creación de una agencia que permita el diálogo directo entre los gobiernos locales y la ONU. En este sentido, la alcaldesa ha señalado que “no podemos solo hablar de planificación si no hablamos de derechos humanos. No podemos soportar que haya ciudades y países en los que el número de personas privadas de derechos son millones”. Carmena además ha subrayado que “es importante que se cree una verdadera cultura de la ciudad. Nos falta cultura de la ciudad”, ha insistido.}}
|date=2017-09-06
|source=[https://diario.madrid.es/blog/notas-d

<div class="gallery-icon landscape"> <a href="https://diario.madrid.es/wp-content/uploads/2017/09/Con-el-alcalde-de-NY-pie.jpg"><img alt="" class="attachment-un_gallery size-un_gallery" height="400" src="https://diario.madrid.es/wp-content/uploads/2017/09/Con-el-alcalde-de-NY-pie-750x400.jpg" width="750"/></a></div>
D:\\tmp\\Con-el-alcalde-de-NY-pie.jpg


The suggested description is:
=={{int:filedesc}}==
{{Information
|description={{es|1=La alcaldesa de Madrid, Manuela Carmena, ha participado hoy en un foro de la Asamblea General de la Organización de Naciones Unidas, para desarrollar la Nueva Agenda Urbana y el papel que tienen las autoridades municipales en este ámbito. Carmena ha reivindicado el papel protagonista de los gobiernos locales en la implementación de la agenda internacional y ha propuesto la creación de una agencia que permita el diálogo directo entre los gobiernos locales y la ONU. En este sentido, la alcaldesa ha señalado que “no podemos solo hablar de planificación si no hablamos de derechos humanos. No podemos soportar que haya ciudades y países en los que el número de personas privadas de derechos son millones”. Carmena además ha subrayado que “es importante que se cree una verdadera cultura de la ciudad. Nos falta cultura de la ciudad”, ha insistido.}}
|date=2017-09-06
|source=[https://diario.madrid.es/blog/notas-d

<div class="gallery-icon landscape"> <a href="https://diario.madrid.es/wp-content/uploads/2017/09/IMG_0026-1339x1000.jpg"><img alt="" class="attachment-un_gallery size-un_gallery" height="400" src="https://diario.madrid.es/wp-content/uploads/2017/09/IMG_0026-750x400.jpg" width="750"/></a></div>
D:\\tmp\\IMG_0026-1339x1000.jpg


The suggested description is:
=={{int:filedesc}}==
{{Information
|description={{es|1=La alcaldesa de Madrid, Manuela Carmena, ha participado hoy en un foro de la Asamblea General de la Organización de Naciones Unidas, para desarrollar la Nueva Agenda Urbana y el papel que tienen las autoridades municipales en este ámbito. Carmena ha reivindicado el papel protagonista de los gobiernos locales en la implementación de la agenda internacional y ha propuesto la creación de una agencia que permita el diálogo directo entre los gobiernos locales y la ONU. En este sentido, la alcaldesa ha señalado que “no podemos solo hablar de planificación si no hablamos de derechos humanos. No podemos soportar que haya ciudades y países en los que el número de personas privadas de derechos son millones”. Carmena además ha subrayado que “es importante que se cree una verdadera cultura de la ciudad. Nos falta cultura de la ciudad”, ha insistido.}}
|date=2017-09-06
|source=[https://diario.madrid.es/blog/notas-d

<div class="gallery-icon landscape"> <a href="https://diario.madrid.es/wp-content/uploads/2017/09/IMG_0031-1339x1000.jpg"><img alt="" class="attachment-un_gallery size-un_gallery" height="400" src="https://diario.madrid.es/wp-content/uploads/2017/09/IMG_0031-750x400.jpg" width="750"/></a></div>
D:\\tmp\\IMG_0031-1339x1000.jpg


The suggested description is:
=={{int:filedesc}}==
{{Information
|description={{es|1=La alcaldesa de Madrid, Manuela Carmena, ha participado hoy en un foro de la Asamblea General de la Organización de Naciones Unidas, para desarrollar la Nueva Agenda Urbana y el papel que tienen las autoridades municipales en este ámbito. Carmena ha reivindicado el papel protagonista de los gobiernos locales en la implementación de la agenda internacional y ha propuesto la creación de una agencia que permita el diálogo directo entre los gobiernos locales y la ONU. En este sentido, la alcaldesa ha señalado que “no podemos solo hablar de planificación si no hablamos de derechos humanos. No podemos soportar que haya ciudades y países en los que el número de personas privadas de derechos son millones”. Carmena además ha subrayado que “es importante que se cree una verdadera cultura de la ciudad. Nos falta cultura de la ciudad”, ha insistido.}}
|date=2017-09-06
|source=[https://diario.madrid.es/blog/notas-d

<div class="gallery-icon landscape"> <a href="https://diario.madrid.es/wp-content/uploads/2017/09/IMG_0029-1339x1000.jpg"><img alt="" class="attachment-un_gallery size-un_gallery" height="400" src="https://diario.madrid.es/wp-content/uploads/2017/09/IMG_0029-750x400.jpg" width="750"/></a></div>
D:\\tmp\\IMG_0029-1339x1000.jpg


The suggested description is:
=={{int:filedesc}}==
{{Information
|description={{es|1=La alcaldesa de Madrid, Manuela Carmena, ha participado hoy en un foro de la Asamblea General de la Organización de Naciones Unidas, para desarrollar la Nueva Agenda Urbana y el papel que tienen las autoridades municipales en este ámbito. Carmena ha reivindicado el papel protagonista de los gobiernos locales en la implementación de la agenda internacional y ha propuesto la creación de una agencia que permita el diálogo directo entre los gobiernos locales y la ONU. En este sentido, la alcaldesa ha señalado que “no podemos solo hablar de planificación si no hablamos de derechos humanos. No podemos soportar que haya ciudades y países en los que el número de personas privadas de derechos son millones”. Carmena además ha subrayado que “es importante que se cree una verdadera cultura de la ciudad. Nos falta cultura de la ciudad”, ha insistido.}}
|date=2017-09-06
|source=[https://diario.madrid.es/blog/notas-d